# Lab | Summarization evaluation using LangSmith
Let's revisit your capstone project 2? Well, sort of. Pick diffierent sets of data and re-run this notebook. Maybe parts of the dataset you used in your last project week. The point is for you to understand all steps involve and the many different ways one can and should evaluate LLM applications using LangSmith.

What did you learn? - Let's discuss that in class

## LangSmith - LangChain evaluation

In [2]:
from google.colab import userdata

OPENAI_API_KEY  = userdata.get('OPENAI_API_KEY')
LANGCHAIN_API_KEY = userdata.get("LANGCHAIN_API_KEY")
HUGGINGFACEHUB_API_TOKEN = userdata.get("HF_TOKEN")

In [4]:
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"]="langsmith_max-test"

In [5]:
#Importing Client from Langsmith
from langsmith import Client
client = Client(api_key=LANGCHAIN_API_KEY)

### Create Dataset


In [10]:
from datasets import load_dataset
cnn_dataset = load_dataset(
    "billsum", split='train'
)

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/91.8M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/15.8M [00:00<?, ?B/s]

data/ca_test-00000-of-00001.parquet:   0%|          | 0.00/6.12M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/18949 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3269 [00:00<?, ? examples/s]

Generating ca_test split:   0%|          | 0/1237 [00:00<?, ? examples/s]

In [17]:
def add_prefix(example):
    return {
        **example,
        "text": f"Summarize this news:\n{example['text']}"
    }

#cnn_dataset = cnn_dataset.map(add_prefix)

In [14]:
cnn_dataset

Dataset({
    features: ['text', 'summary', 'title'],
    num_rows: 18949
})

In [15]:
cnn_dataset[0]

{'text': "SECTION 1. LIABILITY OF BUSINESS ENTITIES PROVIDING USE OF FACILITIES \n              TO NONPROFIT ORGANIZATIONS.\n\n    (a) Definitions.--In this section:\n            (1) Business entity.--The term ``business entity'' means a \n        firm, corporation, association, partnership, consortium, joint \n        venture, or other form of enterprise.\n            (2) Facility.--The term ``facility'' means any real \n        property, including any building, improvement, or appurtenance.\n            (3) Gross negligence.--The term ``gross negligence'' means \n        voluntary and conscious conduct by a person with knowledge (at \n        the time of the conduct) that the conduct is likely to be \n        harmful to the health or well-being of another person.\n            (4) Intentional misconduct.--The term ``intentional \n        misconduct'' means conduct by a person with knowledge (at the \n        time of the conduct) that the conduct is harmful to the health \n        or w

In [18]:
#Get just a few news to test
MAX_NEWS=10
sample_cnn = cnn_dataset.select(range(MAX_NEWS)).map(add_prefix)

sample_cnn

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'summary', 'title'],
    num_rows: 10
})

The dataset contains three columns: article, highlights, and id. To use LangSmith, we need to create a dataset in LangSmith format.

LangSmith expects a prompt and a result. To achieve this, we will transform the article into a prompt by adding the prefix: "Summarize this news." As a result, we will use the content of highlights, which represents the summaries created by humans.

In [19]:
print(sample_cnn[0])

{'text': "Summarize this news:\nSECTION 1. LIABILITY OF BUSINESS ENTITIES PROVIDING USE OF FACILITIES \n              TO NONPROFIT ORGANIZATIONS.\n\n    (a) Definitions.--In this section:\n            (1) Business entity.--The term ``business entity'' means a \n        firm, corporation, association, partnership, consortium, joint \n        venture, or other form of enterprise.\n            (2) Facility.--The term ``facility'' means any real \n        property, including any building, improvement, or appurtenance.\n            (3) Gross negligence.--The term ``gross negligence'' means \n        voluntary and conscious conduct by a person with knowledge (at \n        the time of the conduct) that the conduct is likely to be \n        harmful to the health or well-being of another person.\n            (4) Intentional misconduct.--The term ``intentional \n        misconduct'' means conduct by a person with knowledge (at the \n        time of the conduct) that the conduct is harmful to the

Now We have the Dataset with the prompt and the Reference Summary, it is time to create a Dataset in LangSmith with this information.
### Create the Dataset in Langsmith

The dataset in LangSmith is composed of an input, which is the prompt passed to the model for evaluation, and an output, which should contain what we expect the model to return.

In [20]:
import datetime

In [21]:
import uuid
input_key=['article']
output_key=['highlights']

NAME_DATASET=f"Summarize_dataset_{datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"

In [23]:
#This creates the dataset in LangSmith with the content in sample_cnn - If you run this more than once you will get POST errors
dataset = client.upload_dataframe(
    df=sample_cnn,
    input_keys=['text'],
    output_keys=['summary'],
    name=NAME_DATASET,
    description="Test Embedding distance between model summarizations",
    data_type="kv"
)

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

In this image, we can see an example from the dataset once it's been registered in LangSmith.

In the Input column, there is the prompt to be sent, while in the Output column, the expected output is stored.

When performing the comparison, the model will be given the prompt, and the Cosine distance between its response and the one stored in the sample dataset will be calculated.
<img src="https://github.com/peremartra/Large-Language-Model-Notebooks-Course/blob/main/img/Martra_Figure_4_2SDL_Dataset.jpg?raw=true">

### Recovering Models From Hugging Face
Let's retrieve both models from HuggingFace. A base T5 model and a model that has been fine-tuned using the training portion of this same dataset to generate summaries.

In [26]:
from langchain import HuggingFaceHub

In [27]:
summarizer_base = HuggingFaceHub(
    repo_id="t5-base",
    model_kwargs={"temperature":0, "max_length":180},
    huggingfacehub_api_token=HUGGINGFACEHUB_API_TOKEN
)

/tmp/ipython-input-3938980585.py:1: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpoint``.
  summarizer_base = HuggingFaceHub(


In [28]:
summarizer_finetuned = HuggingFaceHub(
    repo_id="flax-community/t5-base-cnn-dm",
    model_kwargs={"temperature":0, "max_length":180},
    huggingfacehub_api_token=HUGGINGFACEHUB_API_TOKEN
)

## Defining Evaluator
The first step is to define an evaluator, where we specify the variables we want to evaluate. In our case, I have chosen to measure only the "embedding_distance."

I've left the "string_distance" as a comment in case you want to conduct a test with two evaluations instead of one.


In [29]:
from langchain.smith import run_on_dataset, RunEvalConfig
# !pip install -q rapidfuzz==3.6.1

In [30]:
#We are using just one of the multiple evaluator avaiable on LangSmith.

evaluation_config = RunEvalConfig(
    evaluators=[
        "embedding_distance",
        #"string_distance"
    ],
)



### Running Evaluator
With the same configuration, we can launch two evaluations on the same dataset. One for each of the chosen models.

In [33]:
project_name = f"T5-BASE {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"

base_t5_results = run_on_dataset(
    client=client,
    project_name=project_name,
    dataset_name=NAME_DATASET,
    llm_or_chain_factory=summarizer_base,
    evaluation=evaluation_config,
)

View the evaluation results for project 'T5-BASE 2025-09-08 13:38:03' at:
https://smith.langchain.com/o/dbfa5d30-7b5a-4be6-9c3b-7b1d8cb19c7a/datasets/3b28ceeb-33ec-42ab-afe0-a68accb8813b/compare?selectedSessions=c645e2f3-4951-48b0-bec2-771cf9d2cbb8

View all tests for Dataset Summarize_dataset_2025-09-08 13:33:47 at:
https://smith.langchain.com/o/dbfa5d30-7b5a-4be6-9c3b-7b1d8cb19c7a/datasets/3b28ceeb-33ec-42ab-afe0-a68accb8813b
[>                                                 ] 0/10

Error Type: AttributeError, Message: 'InferenceClient' object has no attribute 'post'
Error Type: AttributeError, Message: 'InferenceClient' object has no attribute 'post'


[--------->                                        ] 2/10

Error Type: AttributeError, Message: 'InferenceClient' object has no attribute 'post'
Error Type: AttributeError, Message: 'InferenceClient' object has no attribute 'post'


[------------------->                              ] 4/10

...

[--------------------------------------->          ] 8/10

Error Type: AttributeError, Message: 'InferenceClient' object has no attribute 'post'


[-------------------------------------------->     ] 9/10

Error Type: AttributeError, Message: 'InferenceClient' object has no attribute 'post'


[------------------------------------------------->] 10/10


In [34]:
#Ignore the error shown below
project_name = f"T5-FineTuned {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"

finetuned_t5_results = run_on_dataset(
    client=client,
    project_name=project_name,
    dataset_name=NAME_DATASET,
    llm_or_chain_factory=summarizer_finetuned,
    evaluation=evaluation_config,
)

View the evaluation results for project 'T5-FineTuned 2025-09-08 13:38:17' at:
https://smith.langchain.com/o/dbfa5d30-7b5a-4be6-9c3b-7b1d8cb19c7a/datasets/3b28ceeb-33ec-42ab-afe0-a68accb8813b/compare?selectedSessions=c5449331-8eb1-492b-8de7-0cf7da0e7710

View all tests for Dataset Summarize_dataset_2025-09-08 13:33:47 at:
https://smith.langchain.com/o/dbfa5d30-7b5a-4be6-9c3b-7b1d8cb19c7a/datasets/3b28ceeb-33ec-42ab-afe0-a68accb8813b
[>                                                 ] 0/10

Error Type: AttributeError, Message: 'InferenceClient' object has no attribute 'post'
Error Type: AttributeError, Message: 'InferenceClient' object has no attribute 'post'
Error Type: AttributeError, Message: 'InferenceClient' object has no attribute 'post'


[-------------->                                   ] 3/10

Error Type: AttributeError, Message: 'InferenceClient' object has no attribute 'post'


[------------------->                              ] 4/10

...

[---------------------------------->               ] 7/10

Error Type: AttributeError, Message: 'InferenceClient' object has no attribute 'post'
Error Type: AttributeError, Message: 'InferenceClient' object has no attribute 'post'


[-------------------------------------------->     ] 9/10

Error Type: AttributeError, Message: 'InferenceClient' object has no attribute 'post'


[------------------------------------------------->] 10/10


<img src="https://github.com/peremartra/Large-Language-Model-Notebooks-Course/blob/main/img/Martra_Figure_4_2SDL_Tests.jpg?raw=true">

In the image below you can see the comparision between two tests.
<img src="https://github.com/peremartra/Large-Language-Model-Notebooks-Course/blob/main/img/Martra_Figure_4_2SDL_CompareTestst.jpg?raw=true">

Well, since it has been so straightforward, why don't we try to make the comparison with an OpenAI model?

In [37]:
from langchain_openai import OpenAI
open_aillm=OpenAI(temperature=0.0)

In [38]:
project_name = f"OpenAI {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"

finetuned_t5_results = run_on_dataset(
    client=client,
    project_name=project_name,
    dataset_name=NAME_DATASET,
    llm_or_chain_factory=open_aillm,
    evaluation=evaluation_config,
)

View the evaluation results for project 'OpenAI 2025-09-08 13:39:20' at:
https://smith.langchain.com/o/dbfa5d30-7b5a-4be6-9c3b-7b1d8cb19c7a/datasets/3b28ceeb-33ec-42ab-afe0-a68accb8813b/compare?selectedSessions=025b505b-c1f6-4b76-b221-b24216d21eb4

View all tests for Dataset Summarize_dataset_2025-09-08 13:33:47 at:
https://smith.langchain.com/o/dbfa5d30-7b5a-4be6-9c3b-7b1d8cb19c7a/datasets/3b28ceeb-33ec-42ab-afe0-a68accb8813b
[------------------------>                         ] 5/10

Error Type: BadRequestError, Message: Error code: 400 - {'error': {'message': "This model's maximum context length is 4097 tokens, however you requested 4212 tokens (3956 in your prompt; 256 for the completion). Please reduce your prompt; or completion length.", 'type': 'invalid_request_error', 'param': None, 'code': None}}


[------------------------------------------------->] 10/10


<img src="https://github.com/peremartra/Large-Language-Model-Notebooks-Course/blob/main/img/Martra_Figure_4_2SDL_CompareOpenAI_HF.jpg?raw=true">

The experiment with the OpenAI model has yielded the best results. But, be aware! As we can see, there is a cost involved since we are using an API, and it needs to be paid for.

Another crucial piece of information is that we can view performance data for the models. This data could also be useful for minimally evaluating our inference server.

In [25]:
%pip install -q langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [32]:
import os
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [36]:
%pip install -q langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 3.6 MB/s eta 0:00:00
